In [1]:
#Importamos librerías

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from urllib.request import Request
import requests
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [19]:
#Extraemos datos para paginas 
colonia = []
precios = []
cuartos = []
banos = []
area = []


for i in range(0,2):
    my_url = "https://www.inmuebles24.com/inmuebles-en-venta-en-distrito-federal-pagina-{}.html".format(i)
    
    req = Request(my_url, headers={'User-Agent': 'Mozilla/5.0'})

    uClient = uReq(req)
    page_html = uClient.read()
    uClient.close()

    page_soup2 = soup(page_html2, "html.parser")
    
    
    precio = page_soup2.findAll("span", {"class":"firstPrice"})
    direccion = page_soup2.findAll("span", {"class":"postingCardLocation go-to-posting"})
    features = page_soup2.findAll("ul", {"class":"postingCardMainFeatures go-to-posting"})
    
    for i in range(len(precio)):
        try:
            precios.append(precio[i].text)
        except:
            precios.append(None)

    for i in range(len(direccion)):
        try:
            colonia.append(direccion[i].text)
        except:
            colonia.append(None)
    
    for each in features:
        try:
            cuartos.append((each.find("i", {"class":"postingCardIconsFeatures iconBedrooms"}).next_sibling))
        except:
            cuartos.append(None)  
    
    for each in features:
        try:
            banos.append((each.find("i", {"class":"postingCardIconsFeatures iconBathrooms"}).next_sibling))
        except:
            banos.append(None)  
      
    for each in features:
        try:
            area.append((each.find("i", {"class":"postingCardIconsFeatures iconArea"}).next_sibling))
        except:
            area.append(None)
            

# Dataframe

In [20]:
names = ["precio", "cuartos", "baños", "area", "colonia"]

df = pd.DataFrame(columns=names)
df["precio"] = precios
df["cuartos"] = cuartos
df["baños"] = banos
df["area"] = area
df["colonia"] = colonia

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [21]:
df.head()

,precio,cuartos,baños,area,colonia
0,"\n\t\t\t\t\t\tMN 4,500,000\n\t\t\t\t\t",\n\t\t\t\t\t\t\t\t2 rec.\n\t\t\t\t\t\t\t,\n\t\t\t\t\t2\n\t\t\t\t\t\n\t\t\t\t\t\tbaños\n...,\n\t\t\t\t\t90 m²\n\t\t\t\t,"\n Santa Fe, Alvaro Obregón\n"
1,"\n\t\t\t\t\t\tMN 12,600,000\n\t\t\t\t\t",\n\t\t\t\t\t\t\t\t3 rec.\n\t\t\t\t\t\t\t,\n\t\t\t\t\t3\n\t\t\t\t\t\n\t\t\t\t\t\tbaños\n...,\n\t\t\t\t\t286 m²\n\t\t\t\t,"\n Lomas de Chapultepec, Miguel Hidalgo\n"
2,"\n\t\t\t\t\t\tMN 14,000,000\n\t\t\t\t\t",\n\t\t\t\t\t\t\t\t3 rec.\n\t\t\t\t\t\t\t,\n\t\t\t\t\t3\n\t\t\t\t\t\n\t\t\t\t\t\tbaños\n...,\n\t\t\t\t\t325 m²\n\t\t\t\t,"\n Bosques de las Lomas, Cuajimalpa de Morelos\n"
3,"\n\t\t\t\t\t\tMN 10,850,000\n\t\t\t\t\t",\n\t\t\t\t\t\t\t\t3 rec.\n\t\t\t\t\t\t\t,\n\t\t\t\t\t3\n\t\t\t\t\t\n\t\t\t\t\t\tbaños\n...,\n\t\t\t\t\t217 m²\n\t\t\t\t,"\n Lomas de Vista Hermosa, Cuajimalpa de Morel..."
4,"\n\t\t\t\t\t\tMN 16,000,000\n\t\t\t\t\t",\n\t\t\t\t\t\t\t\t3 rec.\n\t\t\t\t\t\t\t,\n\t\t\t\t\t3\n\t\t\t\t\t\n\t\t\t\t\t\tbaños\n...,\n\t\t\t\t\t305 m²\n\t\t\t\t,"\n Bosques de las Lomas, Cuajimalpa de Morelos\n"


In [22]:
#Limpiamos dataframe

df["precio"] = df["precio"].str.replace("\n","").str.replace(",","").str.replace("\t","").str.replace("MN","").str.replace("USD","").astype(float)

df["colonia"] = df["colonia"].str.replace("\n","")

df["cuartos"] = df["cuartos"].str.extract("(\d+)").astype(int)

df["baños"] = df["baños"].str.extract("(\d+)").astype(int)

df["area"] = df["area"].str.extract("(\d+)").astype(int)

In [23]:
df.head()

,precio,cuartos,baños,area,colonia
0,4500000.0,2,2,90,"Santa Fe, Alvaro Obregón"
1,12600000.0,3,3,286,"Lomas de Chapultepec, Miguel Hidalgo"
2,14000000.0,3,3,325,"Bosques de las Lomas, Cuajimalpa de Morelos"
3,10850000.0,3,3,217,"Lomas de Vista Hermosa, Cuajimalpa de Morelos"
4,16000000.0,3,3,305,"Bosques de las Lomas, Cuajimalpa de Morelos"


In [24]:
#Dividimos colonia y delegacion

new = df["colonia"].str.rsplit(",", n=1, expand=True)

df["delegacion"] = new[1]
df["colonia"] = new[0]

df.drop_duplicates(inplace=True)


In [25]:
df.head()

,precio,cuartos,baños,area,colonia,delegacion
0,4500000.0,2,2,90,Santa Fe,Alvaro Obregón
1,12600000.0,3,3,286,Lomas de Chapultepec,Miguel Hidalgo
2,14000000.0,3,3,325,Bosques de las Lomas,Cuajimalpa de Morelos
3,10850000.0,3,3,217,Lomas de Vista Hermosa,Cuajimalpa de Morelos
4,16000000.0,3,3,305,Bosques de las Lomas,Cuajimalpa de Morelos


In [ ]:
#Guardamos datos

df.to_csv(r"D:\Documentos\UiPath\Datos\Analisis\Inmuebles24.csv", index=False, encoding='utf-8-sig')